# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = '../output_data/cities.csv'

cities = pd.read_csv(path)

cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Henties Bay,-22.1160,14.2845,71.01,77,4,8.37,NaN,1611844859
1,Bethel,41.3712,-73.4140,25.00,68,100,4.00,US,1611844859
2,Saint-François,46.4154,3.9054,55.40,88,90,14.97,FR,1611844859
3,Ushuaia,-54.8000,-68.3000,44.60,76,75,4.61,AR,1611844829
4,Mar del Plata,-38.0023,-57.5575,71.60,73,0,9.22,AR,1611844805


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities[['Lat', 'Lng']]
weights = cities['Humidity'].astype(int)

# Layout
figure_layout = {
     'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights)
# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_vacay = cities.loc[((cities['Max Temp'] >70) & (cities ['Max Temp'] <80)) & 
                        (cities['Wind Speed'] <=10) & (cities['Cloudiness']==0)].dropna()
ideal_vacay

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Mar del Plata,-38.0023,-57.5575,71.60,73,0,9.22,AR,1611844805
5,Kruisfontein,-34.0033,24.7314,77.00,46,0,8.01,ZA,1611844860
59,Hermanus,-34.4187,19.2345,73.00,58,0,5.01,ZA,1611844867
153,Nouadhibou,20.9310,-17.0347,71.91,52,0,3.24,MR,1611844875
198,Abu Dhabi,24.4667,54.3667,73.40,60,0,6.91,AE,1611844889
199,Valle del Guamuez,0.4525,-76.9192,77.00,78,0,2.30,CO,1611844889
203,Sharjah city,25.3573,55.4033,75.20,33,0,5.75,AE,1611844745
227,Los Andes,-32.8337,-70.5983,79.00,79,0,1.01,CL,1611844893
284,Pimentel,-6.8367,-79.9342,71.60,83,0,3.44,PE,1611844900
289,Marzuq,14.4000,46.4667,70.16,29,0,9.51,YE,1611844901


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
ideal = ideal_vacay.copy()
ideal['Hotel Name'] = ""
ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Mar del Plata,-38.0023,-57.5575,71.60,73,0,9.22,AR,1611844805,
5,Kruisfontein,-34.0033,24.7314,77.00,46,0,8.01,ZA,1611844860,
59,Hermanus,-34.4187,19.2345,73.00,58,0,5.01,ZA,1611844867,
153,Nouadhibou,20.9310,-17.0347,71.91,52,0,3.24,MR,1611844875,
198,Abu Dhabi,24.4667,54.3667,73.40,60,0,6.91,AE,1611844889,
199,Valle del Guamuez,0.4525,-76.9192,77.00,78,0,2.30,CO,1611844889,
203,Sharjah city,25.3573,55.4033,75.20,33,0,5.75,AE,1611844745,
227,Los Andes,-32.8337,-70.5983,79.00,79,0,1.01,CL,1611844893,
284,Pimentel,-6.8367,-79.9342,71.60,83,0,3.44,PE,1611844900,
289,Marzuq,14.4000,46.4667,70.16,29,0,9.51,YE,1611844901,


In [6]:
hotels = []

# target_coordinates = list(zip(ideal.Lat, ideal.Lng))
target_radius = 5000
target_type = 'lodging'

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i,row in ideal.iterrows():
    params = {
    'location': (f"{row['Lat']},{row['Lng']}"),
    'radius': target_radius,
    'types': target_type,
    'key': g_key
}
    response = requests.get(base_url, params=params).json()
#     print(response['results'][0]['name'])
    
    try:
        ideal.loc[i,'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print('not found')
        
ideal

not found


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Mar del Plata,-38.0023,-57.5575,71.60,73,0,9.22,AR,1611844805,Gran Hotel Mar del Plata
5,Kruisfontein,-34.0033,24.7314,77.00,46,0,8.01,ZA,1611844860,Oyster Bay House Rental
59,Hermanus,-34.4187,19.2345,73.00,58,0,5.01,ZA,1611844867,Misty Waves Boutique Hotel
153,Nouadhibou,20.9310,-17.0347,71.91,52,0,3.24,MR,1611844875,El Medina
198,Abu Dhabi,24.4667,54.3667,73.40,60,0,6.91,AE,1611844889,Beach Rotana
199,Valle del Guamuez,0.4525,-76.9192,77.00,78,0,2.30,CO,1611844889,Hotel Casa Blanca Orito
203,Sharjah city,25.3573,55.4033,75.20,33,0,5.75,AE,1611844745,Swiss Belhotel Sharjah
227,Los Andes,-32.8337,-70.5983,79.00,79,0,1.01,CL,1611844893,Hotel Los Andes
284,Pimentel,-6.8367,-79.9342,71.60,83,0,3.44,PE,1611844900,Casona Del Muelle
289,Marzuq,14.4000,46.4667,70.16,29,0,9.51,YE,1611844901,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal.iterrows()]
locations = ideal[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

# Create heat layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))